In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from PIL import Image
# import os, sys
# import glob


# IMAGE_EXTENSTION = '.jpg'
# path = '/media/dean/datastore/datasets/darknet/data/construction_set4/coco/images/train2014'
# imgs = glob.glob(os.path.join(path, '*'+IMAGE_EXTENSTION))

# def resize():
#     for img in imgs:
#         if os.path.isfile(img):
#             im = Image.open(img)
#             if im.mode in ("RGBA", "P"):
#                 im = im.convert("RGB")
#             f, e = os.path.splitext(img)
#             #imResize = im.resize((1280,1080), Image.ANTIALIAS)
#             im.save(f.replace('COCO_train2014_0000', '') + '.jpg', 'JPEG', quality=99)
#             print('resized:', img)
#             #!rm -f $img
            
# resize()

In [3]:
import os
import glob
import shutil
import hashlib
from collections import OrderedDict
import pandas as pd
import time
import datetime
import pickle
from utils import datasets
from enum import Enum
import pynmea2

In [4]:
IMAGE_EXTENSTION = '.jpg'

In [5]:

class Format(Enum):
    atm = 0
    ros = 1
    gopro = 2
    raw = 3

In [6]:
# Try on small subset ..
input_format = Format.raw
skip_section = True

## --- Raw Image Data Handler --- ##
if input_format == Format.raw and not skip_section:
    frames_dir = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
    save_dir = os.path.join(frames_dir,'kache_set/')
    os.makedirs(save_dir, 0o755 , exist_ok = True )

    img_data = []
    lookup_table = OrderedDict()
    img_dict = OrderedDict()
    bag_path = os.path.join(frames_dir)

    img_dict['bag_path'] = frames_dir
    img_dict['frames'] = []
    img_dict['csv_path'] = ''

    # Get all images
    imgs = glob.glob(os.path.join(bag_path,'**', '*'+IMAGE_EXTENSTION))
    print(len(imgs))
    for idx, img in enumerate(sorted(set(imgs))):
        print(img)
        d = OrderedDict()
        d['frame_path'] = img
        d['frame_idx'] = int(idx)
        hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()
        shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+IMAGE_EXTENSTION))
        d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+IMAGE_EXTENSTION)
        d['hash_key'] = hex_dig
        d['hash_val'] = hex_val
        d['frame'] = str(d['dataset_path'])
        lookup_table[d['hash_key']] = d['hash_val'] 
        old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

        if d.get('frame', None): img_dict['frames'].append(d)
    if img_dict['frames']: img_data.append((bag_path,img_dict))


## --- ROS Bag Handler --- ##
elif input_format == Format.ros:
    HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
    frames_dir = '/media/dean/datastore/datasets/kache_ai/frames_dev/'
    save_dir = os.path.join(frames_dir,'kache_set/')
    os.makedirs(save_dir, 0o755 , exist_ok = True )

    img_data = []
    lookup_table = OrderedDict()
    img_dict = OrderedDict()
    bag_path = os.path.join(frames_dir)
    img_dict['bag_path'] = frames_dir
    img_dict['frames'] = []
    img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
    csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

    # Get all images
    imgs = glob.glob(os.path.join(bag_path,'**/', '*.JPG'))

    #for idx, img in enumerate(sorted(set(imgs))):
    for idx, img in enumerate(set(imgs)):
        d = OrderedDict()
        d['frame_path'] = img
        d['frame_idx'] = int(idx)
        hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()
        shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
        d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
        d['hash_key'] = hex_dig
        d['hash_val'] = hex_val
        lookup_table[d['hash_key']] = d['hash_val'] 
        old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

        for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
            d['bag_name'] = log[0]
            d['time_sec'] = log[1]
            d['time_readable'] = time.ctime(int(log[1]))

            d['time_nsec'] = log[2]
            d['GPS'] = log[3]
            d['v-ego'] = log[4]

            d['key_event'] = log[5]
            d['frame'] = log[6]

        if d.get('frame', None): img_dict['frames'].append(d)
    if img_dict['frames']: img_data.append((bag_path,img_dict))


        
        
        
        
if not skip_section:        
    pickle_file = os.path.join(save_dir, 'kacheai_logs.pkl')
    pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
    print('Saving to Pickle File:', pickle_file)
    with open(pickle_file,"wb") as pickle_out:
        pickle.dump(pickle_dict, pickle_out)
    print(img_data)



In [7]:
WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'
kache_set = datasets.DataFormatter(annotations_list = os.path.join(WORKING_DIR,'kacheai_logs.pkl'), image_list = WORKING_DIR, input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files/'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

DATA_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/bdd100k.data
MODEL_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/yolov3-bdd100k.cfg
MODEL_WEIGHTS: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/models/yolov3-bdd100k_6327.weights
Initializing Annotation Pipeline via Darknet ... \o/ \o/
Saving to Pickle File: kache_ai_frames_dev_kache_set_kacheai_logs.pkl.pkl
Length of COCO Images 4687
Length of COCO Annotations: 10532
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########

CONSTRUCT-POST | Annotations: 10532  | Images:  3321

################################################



In [ ]:
# WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'

# !aws s3 sync $WORKING_DIR s3://kache-scalabel/kache_ai/frames/

In [9]:
kache_set.export(format = datasets.Format.scalabel, paginate = False)

In [10]:
kache_set.export(format = datasets.Format.scalabel, paginate = True)

KeyError: 'index'

In [7]:
WORKING_DIR = '/media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/'
kache_set_with10cats = datasets.DataFormatter(annotations_list = os.path.join(WORKING_DIR,'kacheai_logs.pkl'), image_list = WORKING_DIR, input_format = datasets.Format.kache,
                                 output_path = os.path.join(WORKING_DIR,'scalabel_files_10cats/'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

DATA_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/bdd100k.data
MODEL_CONFIG: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/cfg/yolov3-bdd100k.cfg
MODEL_WEIGHTS: /media/dean/datastore/datasets/kache_ai/frames_dev/kache_set/scalabel_files/inference/models/yolov3-bdd100k_6327.weights
Initializing Annotation Pipeline via Darknet ... \o/ \o/
Saving to Pickle File: kache_ai_frames_dev_kache_set_kacheai_logs.pkl.pkl
Length of COCO Images 4687
Length of COCO Annotations: 10532
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########

CONSTRUCT-POST | Annotations: 10532  | Images:  3321

################################################



In [ ]:
kache_set.merge(kache_set_with10cats, include=['CONSTRUCT-BARREL', 'trailer',  'CONSTRUCT-CONE','CONSTRUCT-EQUIPMENT', 'CONSTRUCT-POLE', 'CONSTRUCT-POST', 'CONSTRUCT-SIGN', 'person', 'train', 'truck', 'car', 'traffic light', 'traffic light-red','traffic light-amber','traffic light-green', 'bus', 'bike', 'motor', 'traffic sign','traffic sign-slow_sign', 'rider','traffic sign-speed_sign', 'traffic sign-stop_sign'], reject_new_categories = False)

In [ ]:
# input_format = Format.ros

In [ ]:
# ## --- ROS Bag Handler --- ##
# if input_format == Format.ros:

#     HEADER_ROW=['bag', 'time_sec', 'time_nsec', 'GPS', 'v_ego', 'key_event', 'frame_name']
#     frames_dir = '/media/dean/datastore1/datasets/kache_ai/frames/'
#     bag_dirs = os.listdir(frames_dir)
#     save_dir = os.path.join(frames_dir,'kache_set/')
#     os.makedirs(save_dir, 0o755 , exist_ok = True )

#     img_data = []
#     lookup_table = {}
#     # if not os.path.exists(os.path.join())
#     for bag in bag_dirs:
#         if frames_dir != save_dir or '.csv' not in frames_dir:
#             img_dict = {}
#             bag_path = os.path.join(frames_dir, bag)
#             img_dict['bag_path'] = frames_dir
#             img_dict['frames'] = []
#             img_dict['csv_path'] = os.path.join(bag_path, 'frames.csv')
#             if os.path.exists(img_dict['csv_path']):
#                 csv_logger = pd.read_csv(img_dict['csv_path'], names=HEADER_ROW, skiprows=1)

#                 # Get all images
#                 imgs = glob.glob(os.path.join(bag_path,'**/', '*.jpg'))

#                 for img in sorted(set(imgs)):
#                     d = {}
#                     d['frame_path'] = img
#                     hex_val = os.path.abspath(d['frame_path']).replace('/', '^^')+'---'+os.path.abspath(img_dict['csv_path']).replace('/', '^^')
#                     hash_object = hashlib.sha1(str.encode(hex_val))
#                     hex_dig = hash_object.hexdigest()
#                     shutil.copyfile(d['frame_path'], os.path.join(save_dir, str(hex_dig)+'.jpg'))
#                     d['dataset_path'] = os.path.join(save_dir, str(hex_dig)+'.jpg')
#                     d['hash_key'] = hex_dig
#                     d['hash_val'] = hex_val
#                     lookup_table[d['hash_key']] = d['hash_val'] 
#                     old_path = os.path.join(bag_path, 'frames', os.path.split(d['frame_path'])[-1])

#                     for log in [x for x in csv_logger.as_matrix() if os.path.join(bag_path, 'frames', x[6].rstrip('/')) == old_path.rstrip('/')]:
#                         d['bag_name'] = log[0]
#                         d['time_sec'] = log[1]
#                         d['time_readable'] = time.ctime(int(log[1]))

#                         d['time_nsec'] = log[2]
#                         d['GPS'] = log[3]
#                         d['v-ego'] = log[4]
#                         d['latitude'] = None
#                         d['longitude'] = None
#                         if d['GPS']:
#                             msg = pynmea2.parse(d['GPS'])
#                             d['latitude'] = msg.lat
#                             d['longitude'] = msg.lon

#                         d['key_event'] = log[5]
#                         d['frame'] = log[6]

#                     if d.get('frame', None): img_dict['frames'].append(d)
#                 if img_dict['frames']: img_data.append((bag_path,img_dict))

# pickle_file = os.path.join(save_dir, 'kacheai_logs.pickle')
# pickle_dict = {'img_data':img_data,'lookup_table':lookup_table}
# print('Saving to Pickle File:', pickle_file)
# with open(pickle_file,"wb") as pickle_out:
#     pickle.dump(pickle_dict, pickle_out)



In [ ]:
# WORKING_DIR = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/'
# kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/kacheai_logs.pickle', 
#                                    image_list = '/media/dean/datastore/datasets1/kache_ai/frames/kache_set/', 
#                                    input_format = datasets.Format.kache, output_path = os.path.join(WORKING_DIR,'scalabel_files'), use_cache = True,
#                                    trainer_prefix = 'COCO_train2014_0000', s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)

In [ ]:
# kache_set.export(format = datasets.Format.scalabel, paginate = False)

In [ ]:
# kache_set.export(format = datasets.Format.scalabel, paginate = True)

In [ ]:
# !cp /media/dean/datastore1/datasets/kache_ai/frames/kache_set/scalabel_files/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/kache_construction_prelabel_v1.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/bdd_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/bdd100k_train_prelabel.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/bdd_val_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/bdd100k_val_prelabel.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/coco_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/coco_train2014_prelabel.json

In [ ]:
# !cp /media/dean/datastore1/datasets/darknet/data/coco_val_set/bdd100k/annotations/bdd100k_altered_annotations.json /media/dean/datastore1/data-pipeline-master/data_processing/data/fixtures/coco_val2014_prelabel.json

In [ ]:
# !cd /media/dean/datastore1/data-pipeline-master/ && sudo docker-compose up import_json